In [176]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.schema.agent import AgentFinish
from langchain.agents import AgentType, initialize_agent, load_tools

from CIR import CandidateInfoRetreival

In [8]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

In [79]:
jd_ml = """
We are seeking a highly skilled professional for the position of Generative AI Fine Tuning and Proning within our esteemed Computer Software company. This role is pivotal in advancing our generative AI capabilities, ensuring that our models are finely tuned and effectively pruned for optimal performance. The ideal candidate will possess a strong background in machine learning, natural language processing, and deep learning frameworks. You will work collaboratively with our data science and engineering teams, focusing on enhancing the efficiency and effectiveness of generative models. Your expertise in fine-tuning processes and your ability to implement advanced pruning techniques will directly contribute to the deployment of innovative AI solutions. As a member of our team, you will be tasked with exploring state-of-the-art methods and techniques, aiming to lead our projects with cutting-edge research insights. You will be expected to maintain high standards of performance and reliability in our AI systems, thereby driving our commitment to excellence and innovation in the software industry. This position presents an exciting opportunity for those looking to make significant impacts in the field of generative AI, while fostering a culture of continuous improvement and learning within our organization.

Designing, developing, and implementing generative AI models and algorithms utilizing state-of-the-art techniques such as GPT, VAE, and GANs.
Collaborating with cross-functional teams to define AI project requirements and objectives, ensuring alignment with overall business goals.
Conducting research to stay up-to-date with the latest advancements in generative AI, machine learning, and deep learning techniques and identify opportunities to integrate them into our products and services.
Optimizing existing generative AI models for improved performance, scalability, and efficiency.
Developing and maintaining AI pipelines, including data preprocessing, feature extraction, model training, and evaluation.
Developing clear and concise documentation, including technical specifications, user guides, and presentations, to communicate complex AI concepts to both technical and non-technical stakeholders.
Contributing to the establishment of best practices and standards for generative AI development within the organization.
Must be deployed one or two projects
Proven experience of 2+ years
Degree in Computer Science, Artificial Intelligence, Machine Learning, or a related field.
solid experience developing and implementing generative AI models, with a strong understanding of deep learning techniques such as GPT, VAE, and GANs.
proficient in Python and have experience with machine learning libraries and frameworks such as TensorFlow, PyTorch, or Keras.
strong knowledge of data structures, algorithms, and software engineering principles.
familiar with cloud-based platforms and services, such as AWS, GCP, or Azure.
experience with natural language processing (NLP) techniques and tools, such as SpaCy, NLTK, or Hugging Face.
familiar with data visualization tools and libraries, such as Matplotlib, Seaborn, or Plotly.
knowledge of software development methodologies, such as Agile or Scrum.
excellent problem-solving skills, with the ability to think critically and creatively to develop innovative AI solutions.
communication skills, with the ability to effectively convey complex technical concepts to a diverse audience.
possess a proactive mindset, with the ability to work independently and collaboratively in a fast-paced, dynamic environment
"""

jd_projectManager = """
INTERLINK ENGINEERING is a company based in Karachi that specializes in sales and marketing of various solutions including Coding, Inspection, Packaging, Time Attendance, and Security Systems. With branch offices in Lahore, Islamabad, and Faisalabad, the company has a strong team of over 160 people to support customers nationwide. Known for its personalized after-sales support, INTERLINK ENGINEERING is dedicated to serving customers from Karachi to Khyber.

 Role Description
 
This is a full-time on-site role for a Management Coordinator at Interlink Engineering in Lahore. 
The Management Coordinator will be responsible for tasks such as project management, providing customer service, conducting training sessions, and utilizing analytical skills to support business operations.

Qualifications
B.E. Electrical/ Electronics 
Analytical Skills and Project Management Experience
Excellent Communication and Customer Service Skills
Training Abilities
Strong attention to detail and organizational skills
Ability to prioritize tasks and manage time effectively
Experience in the industry is a plus
Bachelor's degree in Business Administration or related field

Experience 
Minimum 10 years both of field as well as office coordination

"""

jd_fullstack = """
SageTeck is looking for web application developers for permanent positions who should be able to design, develop, and maintain web services and web applications.


About the Company: SageTeck is a Pakistan-based software solutions company that provides customized business solutions to clients in international markets.

Degree required: computer science or software engineering

Required Skills:

Experience with PHP MVC Frameworks (Laravel/Symfony)
Experience with at least one of the JS Frameworks like Angular/React/Vue
Experience with OpenSource ERP/CRMs will be a huge plus
Responsibilities:

Design and Implementation of Management Interfaces for Complex Applications
Web-based Dashboard Development
Customization of OpenSource Products
API Creation for Systems Integration
Management Backend Development for Mobile Applications
Benefits:

Working with Cutting-edge Technologies & Up-to-date Techniques
Development of Innovative Ideas from Around the World
Attractive Salary, Bonuses, and Performance Rewards
Health Insurance
Professional Training and Certifications


Experience: 2-3 Years
Salary Range: 250k-350k
Positions: 2
"""

jd = """
We are seeking a highly skilled DevOps Engineer to join our dynamic team. 
The ideal candidate will be responsible for designing, implementing, and maintaining scalable and reliable CI/CD pipelines, automating infrastructure provisioning, and ensuring seamless deployment processes across various environments. As a key member of the team, you will work closely with developers, QA, and operations teams to optimize workflows and enhance system performance.
Lahore candidates will be preferred.
"""


In [136]:
class CV_ScoringSystem:
    
    def __init__(self):
        # Set up model
        self.model = ChatOpenAI(model="gpt-4o-mini")
        
        # parsers
        self.cv_parser = CandidateInfoRetreival()
        self.json_parser = JsonOutputParser()
    
    async def get_candidate_info(self, user_cv):
      return await self.cv_parser.extractInfo(user_cv)

    def scoring_scheme(self):
        scoring_critera = {
                "weights": {
                    "Education": 20,
                    "Experience": 50,
                    "Skills": 30
                },
                "criteria": {
                    "Education": {
                    "Degree": {
                        "max_score": 4,
                        "scoring": {
                        "Bachelor's degree in relevant field": 2,
                        "Master's degree in relevant field": 2,
                        "Bachelor's degree in unrelated field": 0,
                        "No degree": 0
                        }
                    },
                    "Degree Relevance": {
                        "max_score": 2,
                        "scoring": {
                        "Highly relevant degree": 2,
                        "Somewhat relevant degree": 1,
                        "No relevant degree": 0
                        }
                    },
                    "Institution Reputation": {
                        "max_score": 2,
                        "scoring": {
                        "Well-regarded or prestigious institution in relevant field": 2,
                        "Moderately ranked institution in relevant field": 1,
                        "Unknown or low-ranked institution in relevant field": 0
                        }
                    },
                    "Academic Performance": {
                        "max_score": 2,
                        "scoring": {
                        "Excellent academic performance (CGPA above 3.5)": 2,
                        "Average academic performance (CGPA between 2.0 and 3.4)": 1,
                        "Low academic performance or no CGPA provided": 0
                        }
                    }
                    },
                    "Experience": {
                    "Job Relevance": {
                        "max_score": 5,
                        "scoring": {
                        "Directly relevant experience with the same industry, job function, and level of responsibility": 5,
                        "Relevant experience with a similar industry, job function, or level of responsibility": 4,
                        "Partially relevant experience with some transferable skills": 3,
                        "Limited relevance but some transferable skills": 2,
                        "No relevance to the position": 0
                        }
                    },
                    "Years of Experience": {
                        "max_score": 5,
                        "scoring": {
                        "7+ years of relevant experience or more": 5,
                        "5-6 years of relevant experience": 4,
                        "3-4 years of relevant experience": 3,
                        "1-2 years of relevant experience": 2,
                        "Less than 1 year of relevant experience": 1
                        }
                    },
                    "Achievements and Impact": {
                        "max_score": 5,
                        "scoring": {
                        "Consistently achieved exceptional results and made a significant impact": 5,
                        "Frequently achieved above-average results and made a noticeable impact": 4,
                        "Occasionally achieved noteworthy results and made a moderate impact": 3,
                        "Demonstrated some achievements and made a minor impact": 2,
                        "No notable achievements or impact": 0
                        }
                    },
                    "Skills and Expertise": {
                        "max_score": 5,
                        "scoring": {
                        "Possesses an extensive range of relevant skills and expertise": 5,
                        "Demonstrates a broad set of relevant skills and expertise": 4,
                        "Shows a moderate range of relevant skills and expertise": 3,
                        "Exhibits some relevant skills and expertise": 2,
                        "Lacks relevant skills and expertise": 0
                        }
                    },
                    "Progression and Growth": {
                        "max_score": 5,
                        "scoring": {
                        "Consistently demonstrated career progression and took on increasing levels of responsibility": 5,
                        "Experienced noticeable career growth and assumed greater responsibilities": 4,
                        "Showed some career advancement and took on additional responsibilities": 3,
                        "Limited career growth but demonstrated some increased responsibilities": 2,
                        "No career progression or increased responsibilities": 0
                        }
                    }
                    },
                    "Skills": {
                    "Relevance": {
                        "max_score": 2,
                        "scoring": {
                        "The skills and qualifications closely match the requirements of the job": 2,
                        "Some relevant skills and qualifications are mentioned, but there are also irrelevant ones": 1,
                        "The skills and qualifications have little to no relevance to the job": 0
                        }
                    },
                    "Depth of Expertise": {
                        "max_score": 2,
                        "scoring": {
                        "The candidate demonstrates extensive expertise in the mentioned skills and qualifications": 2,
                        "The candidate possesses a moderate level of expertise in the mentioned skills and qualifications": 1,
                        "The candidate lacks depth of expertise in the mentioned skills and qualifications": 0
                        }
                    },
                    "Years of Experience": {
                        "max_score": 2,
                        "scoring": {
                        "The candidate has a significant number of years of experience directly related to the skills and qualifications": 2,
                        "The candidate has some years of experience, but it might not be directly relevant to the skills and qualifications mentioned": 1,
                        "The candidate lacks substantial relevant experience": 0
                        }
                    }
                    }
                }
                }
        
        return scoring_critera

    def extract_weights_max_scores_and_scoring(self, score_template):
      # Extract weights
      weights = score_template.get('weights', {})
      
      # Extract max scores and scoring schemes
      max_scores = {field: 0 for field in weights}  # Initialize max scores for each field
      scoring_schemes = {}
      
      def traverse_criteria(criteria, parent_field=None):
        for key, value in criteria.items():
            if isinstance(value, dict):
                if 'max_score' in value:
                    # Aggregate max_score for the main field
                    if parent_field in max_scores:
                        max_scores[parent_field] += value['max_score']
                    # Add scoring schemes under the main field
                    if parent_field not in scoring_schemes:
                        scoring_schemes[parent_field] = {}
                    scoring_schemes[parent_field][key] = value.get('scoring', {})
                if 'scoring' in value:
                    # Stop at scoring level as it doesn't contain nested fields
                    continue
                traverse_criteria(value, parent_field or key)
      
      traverse_criteria(score_template.get('criteria', {}))
      
      return weights, max_scores, scoring_schemes
    
    def education_score_evaluation(self, candidate_cv, jd_edu, max_score, scoring_schema):
        edu_prompt = ChatPromptTemplate.from_messages([
        ("system", """
         
        You are an expert at evaluating job descriptions against a scoring schema. 
        You will be provided with a job description, candidate's educational information, and a scoring scheme to score it against.
        Your task is to evaluate the provided job description against the CV and output a structured JSON object containing the following field:
        **keys**: 
        - Education Score: This is the total score you gave to the candidate based on the job description
        - Scoring Criteria: This is the thought process you used to give scores to the candidate based on the job description
        
        Scoring Schema:
        {scoring_schema}
        
        Note:
        - You should keep in mind that the scoring should never exceed the max scores defined for that criteria:
        {max_score}
        
        Only output the json data, with nothing else.
                
        """
        ),
        ("user", """
        Job Description: {jd} 
        Candidate Information: {candidate_cv}
        """
        )
    ])

        # Chain the prompt, model, and JSON output parser
        jd_chain = edu_prompt | self.model | self.json_parser
        
        response = jd_chain.invoke({"jd" : jd_edu,
                                    "candidate_cv" : candidate_cv,
                                    "scoring_schema": scoring_schema,
                                    "max_score" : max_score})
        return response
    
    def experience_score_evaluation(self, candidate_cv, jd_exp, max_score, scoring_schema):
        exp_prompt = ChatPromptTemplate.from_messages([
        ("system", """
         
        You are an expert at evaluating job descriptions against a scoring schema. 
        You will be provided with a job description, and the candidate's experience information to score it against.
        Your task is to evaluate the provided job description against the CV and output a structured JSON object containing the following field:
        **keys**: 
        - Experience Score: This is the total score you gave to the candidate based on the job description
        - Scoring Criteria: This is the thought process you used to give scores to the candidate based on the job description
        
        Scoring Schema:
        {scoring_schema}
        
        Note:
        - You should keep in mind that the scoring should never exceed the max scores defined for that criteria:
        {max_score}
        
        Only output the json data, with nothing else.
                
        """
        ),
        ("user", """
        Job Description: {jd} 
        Candidate Information: {candidate_cv}
        """
        )
    ])

        # Chain the prompt, model, and JSON output parser
        jd_chain = exp_prompt | self.model | self.json_parser
        
        response = jd_chain.invoke({"jd" : jd_exp,
                                    "candidate_cv" : candidate_cv,
                                    "scoring_schema": scoring_schema,
                                    "max_score" : max_score})
        return response

    def skill_score_evaluation(self, candidate_cv, jd_skill, max_score, scoring_schema):
        exp_prompt = ChatPromptTemplate.from_messages([
        ("system", """
         
        You are an expert at evaluating job descriptions against a scoring schema. 
        You will be provided with a job description, and the candidate's skill information to score it against.
        Your task is to evaluate the provided job description against the CV and output a structured JSON object containing the following field:
        **keys**: 
        - Skill Score: This is the total score you gave to the candidate based on the job description
        - Scoring Criteria: This is the thought process you used to give scores to the candidate based on the job description
        
        Scoring Schema:
        {scoring_schema}
        
        Note:
        - You should keep in mind that the scoring should never exceed the max scores defined for that criteria:
        {max_score}
        
        Only output the json data, with nothing else.
                
        """
        ),
        ("user", """
        Job Description: {jd} 
        Candidate Information: {candidate_cv}
        """
        )
    ])

        # Chain the prompt, model, and JSON output parser
        jd_chain = exp_prompt | self.model | self.json_parser
        
        response = jd_chain.invoke({"jd" : jd_skill,
                                    "candidate_cv" : candidate_cv,
                                    "scoring_schema": scoring_schema,
                                    "max_score" : max_score})
        return response

In [137]:
ranking = CV_ScoringSystem()

In [154]:
extracted_candidate_info = await ranking.get_candidate_info('./../data/uploaded_pdfs/aqib_latest.pdf')

In [155]:
extracted_candidate_info

{'name': 'Aqib Rehman Pirzada',
 'email': 'aqibrehmanpirzada75@gmail.com',
 'address': 'Islamabad, Pakistan',
 'linkedin_link': 'linkedin.com/AqibRehman',
 'summary': 'Highly skilled Senior ML Engineer with 3+ years of experience designing and deploying scalable ML infrastructure. Proficient in building end-to-end pipelines, optimizing model deployment, and integrating MLOps practices for seamless production workflows. A 2X Kaggle Expert, adept at leveraging tools like FastAPI, Docker, and Cloud to ensure high-performance and reliable ML systems.',
 'experience': [{'job_title': 'Senior AI Engineer',
   'company': 'Qult AI',
   'location': 'Remote, USA',
   'start_date': 'Nov 2024',
   'end_date': 'Till date',
   'responsibilities': ['Designed and implemented an HR AI System for job applicants and HR recruiting agencies, streamlining hiring processes and reducing recruitment time by 40%.',
    'Developed an automated job application feature, which increased application efficiency by 60%

In [156]:
weight_template = ranking.scoring_scheme()

In [157]:
weight_template

{'weights': {'Education': 20, 'Experience': 50, 'Skills': 30},
 'criteria': {'Education': {'Degree': {'max_score': 4,
    'scoring': {"Bachelor's degree in relevant field": 2,
     "Master's degree in relevant field": 2,
     "Bachelor's degree in unrelated field": 0,
     'No degree': 0}},
   'Degree Relevance': {'max_score': 2,
    'scoring': {'Highly relevant degree': 2,
     'Somewhat relevant degree': 1,
     'No relevant degree': 0}},
   'Institution Reputation': {'max_score': 2,
    'scoring': {'Well-regarded or prestigious institution in relevant field': 2,
     'Moderately ranked institution in relevant field': 1,
     'Unknown or low-ranked institution in relevant field': 0}},
   'Academic Performance': {'max_score': 2,
    'scoring': {'Excellent academic performance (CGPA above 3.5)': 2,
     'Average academic performance (CGPA between 2.0 and 3.4)': 1,
     'Low academic performance or no CGPA provided': 0}}},
  'Experience': {'Job Relevance': {'max_score': 5,
    'scoring'

In [158]:
weights, max_scores, scoring_schemes = ranking.extract_weights_max_scores_and_scoring(weight_template)

In [159]:
print(weights) # add checks here
print(max_scores)
print(scoring_schemes)

{'Education': 20, 'Experience': 50, 'Skills': 30}
{'Education': 10, 'Experience': 25, 'Skills': 6}
{'Education': {'Degree': {"Bachelor's degree in relevant field": 2, "Master's degree in relevant field": 2, "Bachelor's degree in unrelated field": 0, 'No degree': 0}, 'Degree Relevance': {'Highly relevant degree': 2, 'Somewhat relevant degree': 1, 'No relevant degree': 0}, 'Institution Reputation': {'Well-regarded or prestigious institution in relevant field': 2, 'Moderately ranked institution in relevant field': 1, 'Unknown or low-ranked institution in relevant field': 0}, 'Academic Performance': {'Excellent academic performance (CGPA above 3.5)': 2, 'Average academic performance (CGPA between 2.0 and 3.4)': 1, 'Low academic performance or no CGPA provided': 0}}, 'Experience': {'Job Relevance': {'Directly relevant experience with the same industry, job function, and level of responsibility': 5, 'Relevant experience with a similar industry, job function, or level of responsibility': 4, '

In [160]:
jd = jd_ml

In [168]:
edu_score = ranking.education_score_evaluation(extracted_candidate_info['education'], jd, max_scores['Education'], scoring_schemes['Education'])

In [169]:
edu_score

{'Education Score': 7,
 'Scoring Criteria': {'Degree': {"Bachelor's degree in relevant field": 2},
  'Degree Relevance': {'Highly relevant degree': 2},
  'Institution Reputation': {'Moderately ranked institution in relevant field': 1},
  'Academic Performance': {'Low academic performance or no CGPA provided': 0}}}

In [170]:
exp_score = ranking.experience_score_evaluation(extracted_candidate_info, jd, max_scores['Experience'], scoring_schemes['Experience'])

In [171]:
exp_score

{'Experience Score': 19,
 'Scoring Criteria': {'Job Relevance': 4,
  'Years of Experience': 3,
  'Achievements and Impact': 4,
  'Skills and Expertise': 4,
  'Progression and Growth': 4}}

In [172]:
skill_score = ranking.skill_score_evaluation(extracted_candidate_info, jd, max_scores['Skills'], scoring_schemes['Skills'])

In [173]:
skill_score

{'Skill Score': 6,
 'Scoring Criteria': {'Relevance': 2,
  'Depth of Expertise': 2,
  'Years of Experience': 2}}

In [174]:
# calculating total score for the candidate:

edu_weight = weights['Education'] * (edu_score['Education Score']/max_scores['Education']) 
exp_weight = weights['Experience'] * (exp_score['Experience Score']/max_scores['Experience']) 
skill_weight = weights['Skills'] * (skill_score['Skill Score']/max_scores['Skills']) 

total_score = edu_weight + exp_weight + skill_weight
print('Canidates Score:', total_score)

Canidates Score: 82.0
